# TiDB Python SDK

A powerful Python SDK for vector storage and retrieval operations with TiDB.

- 🔄 Automatic embedding generation
- 🔍 Vector similarity search
- 🎯 Advanced filtering capabilities
- 📦 Bulk operations support

## Installation

In [19]:
%pip install -q pytidb "pytidb[models]" pandas

Note: you may need to restart the kernel to use updated packages.


## Configuration

- Go [tidbcloud.com](https://tidbcloud.com/) or using [tiup playground](https://docs.pingcap.com/tidb/stable/tiup-playground/) to create a free TiDB database cluster.
  - After creating the cluster, you can find the connection information in the **Connect** tab of your cluster.
  <img width="1512" alt="Image" src="https://github.com/user-attachments/assets/3f9064b7-2060-4d13-a1f3-8e5d1849819b" />
  - Then click "Generate Password" to create a password for your database. And save the connection information including: HOST, PORT, USERNAME, PASSWORD, DATABASE
  <img width="1512" alt="Image" src="https://github.com/user-attachments/assets/8ecd29e5-21df-4dbd-815a-b6cc33f99a69" />
- Go [OpenAI platform](https://platform.openai.com/api-keys) to create your API key.
- Go [jina.ai](https://jina.ai/reranker) to create your Jina API key (For reranker).

Configuration can be provided through environment variables, or using `.env`:

In [ ]:
%%bash

# Check if the .env file is existing.
if [ -f .env ]; then
    exit 0
fi

# Create .env file with your configuration. replace the value with your saved key above.
cat > .env <<EOF
TIDB_HOST=localhost
TIDB_PORT=4000
TIDB_USERNAME=root
TIDB_PASSWORD=
TIDB_DATABASE=test
OPENAI_API_KEY='your_openai_api_key'
JINA_AI_API_KEY='your_jina_api_key'
EOF

In [3]:
import dotenv

dotenv.load_dotenv()

True

## Quickstart

### Connect to TiDB

In [4]:
import os
from pytidb import TiDBClient

db = TiDBClient.connect(
    host=os.getenv("TIDB_HOST"),
    port=int(os.getenv("TIDB_PORT")),
    username=os.getenv("TIDB_USERNAME"),
    password=os.getenv("TIDB_PASSWORD"),
    database=os.getenv("TIDB_DATABASE"),
)

# If you are using DATABASE_URL
# db = TiDBClient.connect(database_url=os.getenv("DATABASE_URL"))

### Create table

In [5]:
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction

# Define your embedding model.
text_embed = EmbeddingFunction("openai/text-embedding-3-small")


class Chunk(TableModel, table=True):
    __tablename__ = "chunks"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    text: str = Field()
    text_vec: list[float] = text_embed.VectorField(
        source_field="text"
    )  # 👈 Define the vector field.
    user_id: int = Field()


table = db.create_table(schema=Chunk)

In [8]:
if not table.has_fts_index("text"):
    table.create_fts_index("text")

### Insert Data

🔢 Auto embedding: when you insert new data, the SDK automatically embeds the corpus for you.

In [9]:
from asyncio import sleep

table.truncate()
await sleep(3)

In [10]:
table.insert(
    Chunk(
        text="TiDB is a distributed database that supports OLTP, OLAP, HTAP and AI workloads.",
        user_id=1,
    ),
)
table.bulk_insert(
    [
        Chunk(
            text="PyTiDB is a Python library for developers to connect to TiDB.",
            user_id=2,
        ),
        Chunk(
            text="LlamaIndex is a framework for building AI applications.", user_id=2
        ),
        Chunk(
            text="OpenAI is a company and platform that provides AI models service and tools.",
            user_id=3,
        ),
    ]
)
table.rows()

4

### Search

#### Vector Search

You don't need to explicitly include all the keywords in your query — vector search uses **semantic similarity** to help you find the most relevant records.

In [11]:
res = (
    table.search(
        "A library for my artificial intelligence software"
    )  # 👈 The query will be embedding automatically.
    .limit(3)
    .to_pandas()
)
res

,id,text,text_vec,user_id,_distance,_score
0,3,LlamaIndex is a framework for building AI appl...,"[-0.008998445, -0.04153499, 0.03558421, -0.035...",2,0.572062,0.427938
1,4,OpenAI is a company and platform that provides...,"[-0.02068534, -0.021361759, 0.024001976, -0.00...",3,0.603195,0.396805
2,2,PyTiDB is a Python library for developers to c...,"[-0.055184152, -0.07363703, -0.0062109437, -0....",2,0.620284,0.379716


#### Fulltext Search

Full-text search tokenizes your query into terms and retrieves the records that mention the most keywords. For example, the query with keyword "library" will retrieve the records that mention the word "library".

In [12]:
res = (
    table.search(
        "A library for my artificial intelligence software", search_type="fulltext"
    )
    .limit(3)
    .to_pandas()
)
res

,id,text,text_vec,user_id,_match_score,_score
0,2,PyTiDB is a Python library for developers to c...,"[-0.055184152, -0.07363703, -0.0062109437, -0....",2,None,None
1,3,LlamaIndex is a framework for building AI appl...,"[-0.008998445, -0.04153499, 0.03558421, -0.035...",2,None,None
2,1,TiDB is a distributed database that supports O...,"[-0.036524396, -0.026345069, 0.06068818, 0.004...",1,None,None


#### Hybrid Search

In [13]:
from pytidb.rerankers import Reranker

jinaai = Reranker(model_name="jina_ai/jina-reranker-m0")

res = (
    table.search(
        "A library for my artificial intelligence software", search_type="hybrid"
    )
    .rerank(jinaai, "text")  # 👈 Rerank the result set with Jina AI reranker.
    .limit(3)
    .to_pandas()
)
res

,id,text,text_vec,user_id,_distance,_score,_match_score
0,3,LlamaIndex is a framework for building AI appl...,"[-0.008998445, -0.04153499, 0.03558421, -0.035...",2,0.572062,0.860416,None
1,4,OpenAI is a company and platform that provides...,"[-0.02068534, -0.021361759, 0.024001976, -0.00...",3,0.603195,0.770188,None
2,1,TiDB is a distributed database that supports O...,"[-0.036524396, -0.026345069, 0.06068818, 0.004...",1,NaN,0.590401,None


### Advanced Filtering

TiDB Client supports various filter operators for flexible querying:

| Operator | Description | Example |
|----------|-------------|---------|
| `$eq` | Equal to | `{"field": {"$eq": "hello"}}` |
| `$gt` | Greater than | `{"field": {"$gt": 1}}` |
| `$gte` | Greater than or equal | `{"field": {"$gte": 1}}` |
| `$lt` | Less than | `{"field": {"$lt": 1}}` |
| `$lte` | Less than or equal | `{"field": {"$lte": 1}}` |
| `$in` | In array | `{"field": {"$in": [1, 2, 3]}}` |
| `$nin` | Not in array | `{"field": {"$nin": [1, 2, 3]}}` |
| `$and` | Logical AND | `{"$and": [{"field1": 1}, {"field2": 2}]}` |
| `$or` | Logical OR | `{"$or": [{"field1": 1}, {"field2": 2}]}` |


In [14]:
chunks = table.query({"user_id": 1})
[(c.id, c.text, c.user_id) for c in chunks]

[(1,
  'TiDB is a distributed database that supports OLTP, OLAP, HTAP and AI workloads.',
  1)]

### Multiple Tables Join

In [16]:
# Create a table to store user data:
class User(TableModel, table=True):
    __tablename__ = "users"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    name: str = Field(max_length=20)


user_table = db.create_table(schema=User)

/Users/liangzhiyuan/Projects/pytidb/.venv/lib/python3.12/site-packages/sqlmodel/main.py:641: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


In [17]:
user_table.truncate()
user_table.insert(User(id=1, name="Alice"))

User(name='Alice', id=1)

In [18]:
from pytidb import Session
from pytidb.sql import select

db_engine = db.db_engine
with Session(db_engine) as db_session:
    query = (
        select(Chunk).join(User, Chunk.user_id == User.id).where(User.name == "Alice")
    )
    chunks = db_session.exec(query).all()

[(c.id, c.text, c.user_id) for c in chunks]

[(1,
  'TiDB is a distributed database that supports OLTP, OLAP, HTAP and AI workloads.',
  1)]

### Update Data

In [21]:
old_chunk = table.query(
    {"text": "PyTiDB is a Python library for developers to connect to TiDB."}
)[0]
chunk_id = old_chunk.id
(old_chunk.text, old_chunk.text_vec)

('PyTiDB is a Python library for developers to connect to TiDB.',
 array([-0.05514111, -0.07355057, -0.00622192, ...,  0.0079887 ,
         0.01695894,  0.02309906], dtype=float32))

In [22]:
table.update(
    values={
        "text": "foo"  # 👈 Only provide the fields you want to update.
    },
    filters={"id": chunk_id},
)

In [23]:
new_chunk = table.get(chunk_id)
(new_chunk.text, new_chunk.text_vec)  # 👈 The vector field is updated automatically.

('foo',
 array([-0.00582053, -0.01312881, -0.0145813 , ..., -0.00910273,
        -0.00738936, -0.01879774], dtype=float32))

### Delete Data

In [24]:
table.rows()

4

In [25]:
table.delete(filters={"user_id": 2})

In [26]:
table.rows()

2

### Execute raw SQL

- Using `execute()` to execute INSERT / UPDATE / DELETE statement.
- Using `query()` to execute SELECT / SHOW statement.

#### Execute SQL

The `execute()` method will return a `SQLExecuteResult` instance.

In [27]:
db.execute("INSERT INTO chunks(text, user_id) VALUES ('inserted from raw sql', 5)")

SQLExecuteResult(rowcount=1, success=True, message=None)

Both of the `execute` and `query` methods are support the **Parameterized SQL** feature, which help you avoid [SQL injection](https://en.wikipedia.org/wiki/SQL_injection) while using the dynamic SQL feature.

In [28]:
db.execute(
    "INSERT INTO chunks(text, user_id) VALUES (:text, :user_id)",
    {"text": "inserted from dynamic sql", "user_id": 6},
)

SQLExecuteResult(rowcount=1, success=True, message=None)

#### Query SQL

The `query()` method will return a `SQLQueryResult` instance with some helper methods:

- `to_pandas()`
- `to_list()`
- `to_rows()`
- `scalar()`

The `to_pandas()` method to convert the query result to a `pandas.DataFrame`, which is displayed as human-friendly style on the notebook:

In [29]:
db.query("SELECT id, text, user_id FROM chunks").to_pandas()

,id,text,user_id
0,1,TiDB is a distributed database that supports O...,1
1,4,OpenAI is a company and platform that provides...,3
2,30001,inserted from raw sql,5
3,30002,inserted from dynamic sql,6


The `to_list()` method will convert the query result into a list of dict.

In [30]:
db.query(
    "SELECT id, text, user_id FROM chunks WHERE user_id = :user_id", {"user_id": 3}
).to_list()

[{'id': 4,
  'text': 'OpenAI is a company and platform that provides AI models service and tools.',
  'user_id': 3}]

The `to_rows` method will return a list of tuple, every tuple represent of one row of data.

In [31]:
db.query("SHOW TABLES;").to_rows()

[('chunks',), ('users',)]

In [32]:
db.query("SELECT COUNT(*) FROM chunks;").scalar()

4

### Table Management

#### List tables

In [33]:
db.table_names()

['chunks', 'users']

#### Truncate table

Clear all data in the table:

In [34]:
table.truncate()
table.rows()

0

#### Drop table


In [35]:
db.drop_table("chunks")

SQLExecuteResult(rowcount=0, success=True, message=None)